In [1]:
# check if running in AWS
import os 
user= os.environ.get("USER")
IS_AWS = (user is not None and "ec2" in user) and True or False 
IS_AWS

False

In [2]:
# local function 
def local_code():
    pass

In [3]:
# aws function 
def aws_code(): 
    pass

In [4]:
# main 
if __name__ == '__main__': 
    result = IS_AWS and aws_code() or local_code()